_Примечание: в контексте данной работы единицами рекомендаций будут являвляться фильмы, т.к. использование этого термина будет удобно с точки зрения используемого датасета._

Можно выделить два основных типа рекомендательных систем.

# Разработка рекомендательной системы на Python

**Content-based**

* Пользователю рекомендуются фильмы, похожие на те, которые он уже посмотрел.
* Похожести оцениваются по признакам содержимого объектов.
* Сильная зависимость от предметной области, полезность рекомендаций ограничена.

Коллаборативная фильтрация (**Collaborative Filtering**)

* Для рекомендации используется история оценок как самого пользователя, так и других пользователей.
* Более универсальный подход, часто дает лучший результат.
* Есть свои проблемы (например, холодный старт).

В большинстве случаев алгоритмы коллаборативной фильтрации (CF) показывают лучший результат, чем content-based системы. В данной работе будут рассматриваться два типа CF: **Memory-Based Collaborative Filtering** и **Model-Based Collaborative filtering**.

## Датасет

В данной работе используется MovieLens Dataset (Small). Посмотреть информацию или скачать датасет можно [отсюда](https://grouplens.org/datasets/movielens/).

In [70]:
import requests
import uuid             
#UUID = uuid.uuid4().hex
#print(UUID)
URL_BEGIN_DATA = 'http://127.0.0.1:8000/predict'
req = requests.post(URL_BEGIN_DATA,json={"id_cl":300})
req.json()
#URL_TASK_RESULT_GET='http://127.0.0.1:8001/'
#response = requests.get(URL_TASK_RESULT_GET.format(uuid=UUID))
#response.json()

{'predictions': [232,
  20,
  16,
  28,
  192,
  7,
  463,
  34,
  166,
  3,
  480,
  26,
  4,
  32,
  444]}

In [77]:
URL_BEGIN_DATA = 'http://127.0.0.1:8000/train'

req = requests.post(URL_BEGIN_DATA,json={"train":1})
req.json()
#req.stat()

AttributeError: 'Response' object has no attribute 'stat'

In [5]:
import numpy as np
import pandas as pd

In [6]:
# загружаем датасет
df = pd.read_csv('../data/ml-latest-small/ratings.csv')

In [7]:
# смотрим на структуру
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [39]:
# выводим количество пользователей и фильмов
n_users = df['userId'].unique().shape[0]
n_items = df['movieId'].unique().shape[0]

In [40]:
df['rating'].unique()

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

In [9]:
print('Users: {}, Items: {}'.format(n_users, n_items))

Users: 610, Items: 9724


In [10]:
# чтобы можно было удобно работать дальше, необходимо отмасштабировать 
# значения в колонке movieId (новые значения будут в диапазоне от 1 до
# количества фильмов)
input_list = df['movieId'].unique()

def scale_movie_id(input_id):
    return np.where(input_list == input_id)[0][0] + 1

df['movieId'] = df['movieId'].apply(scale_movie_id)

In [33]:
from sklearn.model_selection import train_test_split

# делим данные на тренировочный и тестовый наборы
train_data, test_data = train_test_split(df, test_size=0.01)

## Memory-Based Collaborative Filtering

Memory-Based Collaborative Filtering подходы можно разделить на две части: **user-item filtering** and **item-item filtering**.

В user-item filtering мы:

1. Берём исходного пользователя
2. Находим группу пользователей, которая максимально похожа на него (основываясь, например, оценках) и узнаём, какие фильмы понравились этой группе.
3. Нашему исходному пользователю рекомендуем фильмы, которые нравятся найденной группе пользователей.

На входе пользователь, на выходе – рекомендация фильмов для данного пользователя.

В item-item filtering мы:
    
1. Берём какой-либо фильм
2. Находим пользователей, которым нравится этот фильм
3. Смотрим на фильмы, которые нравятся найденным пользователям и выводим их в качестве рекомендации к исходному товару

На входе фильм, на выходе – рекомендация в виде похожих фильмов.

* Item-Item Collaborative Filtering: "Пользователям, которым нравится данный фильм, может так же понравиться это ..."
* User-Item Collaborative Filtering: "Похожим на вас пользователям нравится это ..."

В обоих случаях нам необходимо создать user-item матрицу, которая будет выглядеть следующим образом:

|       | Item1 | Item2 | Item3 |
|-------|-------|-------|-------|
| User1 |   5   |   3   |   4   |
| User2 |   4   |   0   |   0   |
| User3 |   0   |   0   |   0   |

В ячейках матрицы будет записана информация об оценке фильма $m$ пользователя $n$.

In [22]:
train_data

,userId,movieId,rating,timestamp
23438,160,1539,1.0,976797780
63144,414,1619,3.0,961596684
32263,221,296,4.5,1111176234
42625,288,2922,2.0,975691939
6854,45,876,5.0,1057006897
...,...,...,...,...
94979,599,9279,2.0,1519327922
90270,587,310,4.0,953138160
81446,514,8646,0.5,1536381303
735,6,315,5.0,845553726


In [12]:
train_data_matrix = train_data.pivot(index='userId', 
                                                  columns='movieId', 
                                                  values='rating').fillna(0)
np.asarray(train_data_matrix)

array([[0. , 4. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 2. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 5. , ..., 3. , 3.5, 3.5]])

In [13]:
train_data_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,9714,9715,9716,9717,9719,9720,9721,9722,9723,9724
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,4.0,4.0,5.0,5.0,3.0,5.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,3.0,0.0,4.0,0.0,3.5,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,0.0,4.5,4.5,3.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
test_data_matrix.shape

(610, 9724)

In [36]:
test_data_matrix = np.zeros((n_users, n_items))

In [37]:
test_data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
# создаём две user-item матрицы – для обучения и для теста
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1] - 1, line[2] - 1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    print(line[1])
    test_data_matrix[line[1] - 1, line[2] - 1] = line[3]

#df_data_matrix = df.pivot(index='userId', 
#                                                  columns='movieId', 
#                                                  values='rating').fillna(0)
#df_matrix = np.zeros((n_users, n_items))
#for line in df_data_matrix.itertuples():
#    df_matrix[line[1] - 1, line[2] - 1] = line[3]

84
484
346
42
232
226
298
63
27
332
166
599
438
486
344
52
495
385
93
84
489
477
517
234
122
10
345
87
200
474
603
148
452
448
448
571
534
50
169
610
562
414
185
199
599
129
177
156
292
555
222
606
600
182
387
288
17
414
220
541
28
298
45
332
414
380
128
186
15
587
204
136
338
74
410
480
387
266
18
82
140
95
418
7
182
83
442
307
339
382
195
559
599
483
555
181
448
561
287
534
414
438
93
226
288
509
121
125
156
490
596
412
281
542
323
332
68
19
232
247
483
265
489
462
606
186
288
340
290
234
462
249
247
414
270
44
474
474
298
448
504
286
600
239
221
517
354
432
125
119
249
562
503
356
137
372
600
483
599
610
438
338
480
160
526
270
580
474
483
558
4
199
113
298
585
332
4
385
380
395
602
517
72
307
274
448
122
535
148
217
514
28
210
377
171
82
50
274
219
274
288
368
135
274
19
524
387
219
173
279
1
152
397
50
573
368
489
602
256
606
489
573
21
323
28
477
489
199
334
343
538
529
430
232
414
232
298
414
487
356
387
521
537
66
414
554
608
89
304
21
64
38
191
199
599
185
307
608
168
244
198


In [24]:
train_data_matrix

array([[0. , 4. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 2. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 5. , ..., 3. , 3.5, 3.5]])

После того, как мы построим данную матрицу, на её основе необходимо будет рассчитать две новые матрицы с коэффициентами сходства (похожести, близости) для пользователей и для фильмов.

В качестве метрики близости в данной работе используется косинусное расстояние между векторами пользователей (фильмов).

Формула для пользователей:

$$ s_{u}^{cos}(u_k, u_a) = \frac{u_k \cdot  u_a}{\left \| u_k \right \| \left \| u_a \right \|} = \frac{\sum x_{k,m} x_{a,m}}{\sqrt{\sum x_{k,m}^2 \sum x_{a,m}^2}} $$

Формула для фильмов:

$$ s_{u}^{cos}(i_m, i_b) = \frac{i_m \cdot  i_b}{\left \| i_m \right \| \left \| i_b \right \|} = \frac{\sum x_{a,m} x_{a,b}}{\sqrt{\sum x_{a,m}^2 \sum x_{a,b}^2}} $$

In [21]:
test_data_matrix.shape

(610, 9724)

In [9]:
from sklearn.metrics.pairwise import pairwise_distances

# считаем косинусное расстояние для пользователей и фильмов
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

Матрица "похожести" для пользователей имеет следующий вид (аналогичную структуру имеет и матрицы для фильмов):

|       | User1 | User1 | User3 |
|-------|-------|-------|-------|
| User1 |   0   | 0.87  | 0.99  |
| User2 |   123   |   0   |   123123   |
| User3 |   123   |   123   |   0  |

Для предсказания в user-based CF необходимо применить следующую формулу:

$$ \hat{x}_{k,m} = \overline{x}_k + \frac{\sum_{u_a} sim_u(u_k, u_a)(x_{a,m} - \overline{x}_{u_a})}{\sum_{u_a} \left | sim_u(u_k, u_a) \right |} $$

Для item-based CF:

$$ \hat{x}_{k,m} = \frac{\sum_{i_b} sim_i(i_m, i_b)(x_{k,b})}{\sum_{i_b} \left | sim_i(i_m, i_b) \right |} $$

In [55]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [56]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

Для оценки качества предсказания используем метрику RMSE (Root Mean Square Error, cреднеквадратичная ошибка):

$$ RMSE = \sqrt{\frac{1}{N}\sum (x_i - \hat{x}_i)^2} $$

In [62]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [63]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

NameError: name 'user_prediction' is not defined

## Model-based Collaborative Filtering

Model-based Collaborative Filtering основана на разложении матрицы. В данной работе используется метод разложения, который называется **singular value decomposition** (SVD, cингулярное разложение). Смысл этого разложения в том, что исходную матрицу $X$ мы разбиваем на произведение ортогональных матриц $U$ и $V^T$ и диагональной матрицы $S$.

$$ X = UV^TS $$

В нашем случае $X$ – разреженная (состоящая преимущественно из нулей) user-item матрица. Разложив её на компоненты, мы можем их вновь перемножить и получить "восстановленную" матрицу $\hat{X}$. Матрица $\hat{X}$ и будет являться нашим предсказанием – метод SVD сделал сам за нас всё работу и заполнил пропуски в исходной матрице $X$

$$ UV^TS \approx \hat{X}$$

In [50]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

In [19]:
train_data_matrix_ = train_data.pivot(index='userId', 
                                                  columns='movieId', 
                                                  values='rating').fillna(0)
np.asarray(train_data_matrix_).shape

(610, 8971)

In [20]:
test_data_matrix_ = test_data.pivot(index='userId', 
                                                  columns='movieId', 
                                                  values='rating').fillna(0)
np.asarray(test_data_matrix_).shape

(607, 5117)

In [68]:
train_data_matrix_

movieId,1,2,3,4,5,6,7,8,9,10,...,9714,9715,9716,9717,9718,9719,9721,9722,9723,9724
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,4.0,0.0,0.0,5.0,3.0,0.0,4.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,3.0,4.5,4.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,0.0,4.5,4.5,3.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
import sklearn.decomposition as skd


In [41]:
model_trsvd = skd.TruncatedSVD(n_components=15)
model = model_trsvd.fit(train_data_matrix)

In [48]:
model.transform(test_data_matrix)[600]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [79]:
cf_preds_df = pd.DataFrame(test_data_matrix, columns = test_data_matrix_.columns).transpose()


In [54]:
# делаем SVD
u, s, vt = svds(train_data_matrix, k=10)
s_diag_matrix = np.diag(s)

# предсказываем
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)

# выводим метрику
#print('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix)))

In [55]:
import pickle

In [67]:
pickle.dump(dict(svds),open('fe.pkl','wb'))

TypeError: 'function' object is not iterable

In [64]:
sv=pickle.load(open('fe.pkl','rb'))

EOFError: Ran out of input

In [16]:
test_data_matrix.nonzero()

(array([  0,   0,   0, ..., 609, 609, 609]),
 array([   0,    2,    3, ..., 9700, 9703, 9719]))

In [97]:
X_pred.shape

(610, 9724)

## Hybrid Recommender Systems

Пару слов стоит сказать о наиболее эффективной на практике гибридной рекомендательной системе. Его суть заключается в том, чтобы комбинировать в себе сontent-based модели и сollaborative filtering. Используя дополнительную информацию о фильмах (жанр, теги, год выпуска и т.д.) и мощь алгоритмов коллаборативной фильтрации, можно добиться впечатляющего результата.

## Что почитать по этой теме

[Implementing your own recommender systems in Python](http://online.cambridgecoding.com/notebooks/eWReNYcAfB/implementing-your-own-recommender-systems-in-python-2)<br/>
[Как работают рекомендательные системы. Лекция в Яндексе](https://habrahabr.ru/company/yandex/blog/241455/)